# <font color='blue'>Uni-Facef - Data Science - Parte 1 </font>

    Neste notebook vamos criar um modelo de classificação usando o algorítmo Gradient Boosting. Vamos fazer o treinamento do modelo, medir sua acurácia (nível de acerto) e fazer a persistência (serialização do objeto em arquivo)

    Vamos utilizar a biblioteca Pandas e Scikit Learn.

### GRADIENT BOOSTING

    Trata-se de um Método Ensemble, onde ocorre a predição de múltiplos algoritmos base de árvore de regressão em um único classificador. Esse algoritmo implementa as seguintes técnicas:


#### Boosting
	
    Realiza a classificação a partir de um sistema de peso, constituído a partir do resultado de classificação de cada algoritmo base.

#### Gradient Descent
	
    Otimização iterativa para encontrar o mínimo de uma função de custo, a Cost function.


## Lendo o dataset em Dataframe Pandas 

In [ ]:
import pandas as pd

# Lendo o dataset em csv e criando um Dataframe Pandas
df = pd.read_csv("gradient_boosting_training.csv", delimiter="|")

df.head(20)

## Verificando a distribuição entre as classes (variável target)

É importante termos as classes bem balanceadas para que o algorítmo aprenda a relação desses dados de forma mais assertiva

In [ ]:
print(df.groupby('propenso').size())

## Fazendo um pré-processamento dos dados

Em sua grande maioria das vezes o trabalho de pré-processamento é pesado, pois podem haver muitos campos nulos, com ruídos (outliers), variáveis categóricas da qual muitos algorítimos não trabalham, etc. E há muitas técnicas para que cada caso possa ser tratado.

Nesse exemplo, consideramos que fizemos um trabalho de processamento e padronização previamente pelo Spark teremos um menor esforço de tratamento.

In [ ]:
import numpy as np

# Tratando dados missing no vlrmediocompra
df['vlrmediocompra'] = df['vlrmediocompra'].replace(np.NaN, 0)

# Vamos usar a técnica de colocar média da massa para a idade
df['idade'].fillna(df['idade'].mean(), inplace=True)

# Vamos usar a média da massa para a recencia também
df['recencia'].fillna(df['recencia'].mean(), inplace=True)

## Separando a variável target (resposta) das variáveis preditoras

- X - Variáveis preditoras
- Y - Variável target ou resposta

In [ ]:
cols = [col for col in df.columns if col not in ['id','propenso', 'recencia', 'scorecompra']]
print(cols)

In [ ]:
"""
Arrays:
 - X - Variáveis preditoras
 - Y - Variável target ou resposta
"""
X = df[cols].values
Y = df.propenso.values

## Colocando os valores das variável preditoras numa mesma escala

Observe que os valores numéricos não estão no mesma escala e muitos algorítimos terão melhor performance fazendo essa normalização

In [ ]:
df.head(5)

#### Para isso usaremos o método "scale" do módulo "preprocessing" do "sklearn"

In [ ]:
from sklearn.preprocessing import scale

X = scale(X)

print(X)

## Separando os dados de treino dos dados de teste

Para isso usaremos o método "train_test_split" do módulo "model_selection" do "sklearn"

In [ ]:
from sklearn.model_selection import train_test_split

# Percentual da massa para treino
validation_size = 0.30

"""
Retorna 4 arrays diferentes:

X_Train - Variáveis preditoras para o treinamento do modelo
X_Test - Variáveis preditoras para o teste do modelo

Y_Train - Variável target ou resposta para treinamento do modelo
Y_Test - Variável target ou resposta usada para validação do modelo
"""

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = validation_size)

In [ ]:
Y_Train

## Criando uma instancia do algoritmo Classificador

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(
    n_estimators=50,
    learning_rate=1, 
    min_samples_split=3, 
    min_samples_leaf=2)

print(clf)

## Treinamento e persistência do modelo

Treina o modelo através do método "fit" do Classificador

In [ ]:
model = clf.fit(X_Train, Y_Train)

#### Persistirá o modelo treinado em arquivo

Para isso usaremos o módulo "pickle" que fará a serialização do objeto

In [ ]:
import pickle

pickle.dump(model, open("gradient_boosting_model_pickle.dat", "wb"))

## Fazendo a predição da variável resposta esperada

Podemos utilizar o modelo em memória ou fazer o "load" do modelo persistido

In [ ]:
model_serial = pickle.load(open("gradient_boosting_model_pickle.dat", "rb"))

model_serial

#### Fazendo previsão com variaveis preditoras de teste através do método "predict" do classificador

In [ ]:
prediction = model.predict(X_Test)

prediction

In [ ]:
score = model.predict_proba(X_Test)[:,1]

score

In [ ]:
df_predict = pd.DataFrame(prediction, columns=['propenso'])
df_score = pd.DataFrame(score, columns=['score'])

df_result = pd.merge(df_predict, df_score, how = 'left', left_index = True, right_index = True)

df_result.head(10)

## Avaliando o modelo

In [ ]:
from sklearn.metrics import accuracy_score

print (accuracy_score(Y_Test, prediction))

In [ ]:
from sklearn.metrics import confusion_matrix

print ("Confusion Matrix:\n")
print (confusion_matrix(Y_Test, prediction))

In [ ]:
from sklearn.metrics import classification_report

print ("Classification Report:\n")
print (classification_report(Y_Test, prediction))

### Relevância das principais variáveis preditoras para o Modelo 

In [ ]:
model_serial.feature_importances_

In [ ]:
cols

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure()
plt.title("Variáveis mais importantes")

plt.bar(range(X.shape[1]), importances[indices], color="b", align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])

plt.show()

# Mostrar legenda abaixo do gráfico
for i in range(0,len(cols)):
    print (str(i)+' - '+ cols[i] +' --> '+ str(model.feature_importances_[i]))

### Verificando a correlação entre as variáveis

In [ ]:
df.corr()

#### Aprenda mais sobre "scikit-learn": 

https://scikit-learn.org/stable/